# 1

In [1]:
import numpy as np

In [5]:
animals = ['Cow', 'Dolhpin', 'Chicken', 'Seal', 'Penguin', 'Bat']

hypotheses = [
    [1,1,0,1,0,1], # Viviparous (definitionally all mammals in set)
    [0,0,1,0,1,0], # Oviparous
    [0,0,1,0,1,1], # Winged
    [0,0,1,0,1,0], # Winged non-flying
    [1,0,1,0,1,1], # Separate 2 hind limbs
    [0,1,0,1,1,0], # Marine habitat
    [1,0,1,0,0,0], # Commonly found on farms
    [0,0,0,1,1,0], # Commonly found in cold areas
    [0,1,0,1,1,0], # Carnivorous
    [1,0,0,0,0,0], # Herbivorous
    [0,0,1,0,0,1], # Omnivorous
    [1,1,1,1,1,1], # Animals
]

hypotheses = np.array(hypotheses)
# display(hypotheses)

hypotheses_names = ['Viviparous', 'Oviparous', 'Winged', 'Winged non-flying', 'Separate 2 hind limbs', 'Marine habitat', 'Commonly found on farms', 'Commonly found in cold areas', 'Carnivorous', 'Herbivorous', 'Omnivorous', 'Animals']
hypotheses_named = {name: hypothesis for name, hypothesis in zip(hypotheses_names, hypotheses)}

# display(hypotheses_named)

# 2

It is plausible to expect that, when coming up with hypotheses, I may have created more hypotheses for certain subsets of animals simply because they came to mind more readily. In the extreme, I could include many copies of nearly-identical hypotheses despite the fact that they may not be representative of all plausible hypotheses.

A prior distribution I think may heuristically address this concern is to have hypothesis $h_i = [a_{i1}, a_{i_2}, \cdots, a_{i6}]$ have a prior likelihood of $\prod_{j \in [1,6], a_{ij} = 1} \frac{1}{n(j)}$, where $n(j)$ is the total number of times animal $i$ appears in hypotheses. This likelihood is normalized such that the sum of all hypotheses' prior likelihoods adds to 1. Note how this prior will not yield undefined results because an animal which appears in 0 hypotheses (hence $n(j) = 0$) will have an $a_{ij} = 0$.

In [7]:
n_j = np.sum(hypotheses, axis=1)
display(n_j)

array([4, 2, 3, 2, 4, 3, 2, 2, 3, 1, 2, 6])

# 3

# 4

# 5